In [19]:
import time
import pandas as pd
import pickle
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

# 데이터 로드

In [20]:
import pickle
with open('df_sector.pickle','rb') as f:
    df_sector = pickle.load(f)

In [21]:
with open('kospi.pickle','rb') as f:
    kospi = pickle.load(f)

# 1. 등락률 계산
# 2. 거래변동량 계산

In [22]:
#등락률 계산 : 일 등락률 = (오늘종가 – 어제종가) / 어제종가 * 100 => 평균
#등락률
for sector_name, df_dict in df_sector.items():
    for stockcode, df in df_dict.items():
        등락률 = []
        거래변동량 = []
        for i in range(len(df)): #현재가 수
            if i == len(df)-1: #각 항목의 마지막 인덱스인경우
                break
            else:    
                등락률.append((df['종가'][i]-df['종가'][i+1])/df['종가'][i+1]*100)
                거래변동량.append(abs((df['거래량'][i]-df['거래량'][i+1])))
        등락률.append(0)
        거래변동량.append(0)
        df['등락률']=등락률
        df['거래변동량']=거래변동량
        del df['거래량']

# 3. 데이터가 부족한 항목 조회

In [23]:
lack_sector_list={}
for sector_name, df_dict in df_sector.items():
    lack_stock_list = []
    for stockcode, df in df_dict.items():
        if len(df)<3000:
            lack_stock_list.append(stockcode)
    lack_sector_list[sector_name] = lack_stock_list
print(lack_sector_list)
            
            
            

{'Food': ['271560'], 'Clothing': ['383220'], 'Chemical': [], 'Medicine': ['207940', '302440', '128940'], 'Non_Metal': ['300720'], 'Metal': [], 'Machine': ['241560', '112610'], 'Electronic': ['373220'], 'Construction': [], 'Transport': ['180640'], 'Distribution': ['028260', '282330', '139480'], 'Power': [], 'Tele': [], 'Finance': ['323410', '377300'], 'Brokerage': [], 'Insurer': [], 'Service': ['259960', '018260'], 'Manufacturer': ['329180']}


# 4. 업종별 데이터프레임 합치기

In [24]:
df_concat_sector = {}
for sector_name, df_dict in df_sector.items():
    sector_df = pd.concat(df_dict,axis=0)
    sector_df.reset_index(drop=True, inplace=True)
    df_concat_sector[sector_name] = sector_df
    

In [25]:
df_concat_sector['Food'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13232 entries, 0 to 13231
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일자      13232 non-null  datetime64[ns]
 1   종가      13232 non-null  int32         
 2   등락률     13232 non-null  float64       
 3   거래변동량   13232 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(1)
memory usage: 361.9 KB


In [26]:
df_mean_sector = {}
for sector_name, sector_df in df_concat_sector.items():
    df_mean = sector_df.groupby(['일자'],as_index=False).mean()
    df_mean_sector[sector_name] = df_mean
    

In [27]:
df_mean_sector['Food']

,일자,종가,등락률,거래변동량
0,2010-05-14,135837.5,0.000000,0.00
1,2010-05-17,136225.0,-0.473023,26597.00
2,2010-05-18,140650.0,1.299365,43176.25
3,2010-05-19,138175.0,-1.185370,28548.00
4,2010-05-20,138462.5,-0.102268,29688.25
...,...,...,...,...
2995,2022-07-06,197660.0,0.165308,24833.00
2996,2022-07-07,197790.0,0.501700,41801.00
2997,2022-07-08,196530.0,-0.394951,23997.00
2998,2022-07-11,195770.0,-0.759630,16710.00


# 5. 코스피 증감량 계산

In [28]:
코스피지수_증감량 = []
for i in range(len(kospi)):
    if i == len(kospi)-1:
        break
    else:    
        코스피지수_증감량.append((kospi['코스피지수'][i]-kospi['코스피지수'][i+1]))
코스피지수_증감량.append(0)
kospi['코스피지수_증감량']=코스피지수_증감량
kospi.drop(['코스피지수'],axis=1,inplace=True)
kospi

,일자,코스피지수_증감량
0,2022-07-12,-22.51
1,2022-07-11,-10.34
2,2022-07-08,16.34
3,2022-07-07,42.26
4,2022-07-06,-49.77
...,...,...
2995,2010-05-20,-29.90
2996,2010-05-19,-13.16
2997,2010-05-18,-8.27
2998,2010-05-17,-44.12


# 6. 섹션별 데이터와 코스피 지수 합치기

In [29]:
result = {}
for sector_name, df in df_mean_sector.items():
    result[sector_name] =pd.merge(df,kospi,on='일자',how='left')
    

In [30]:
result['Food']

,일자,종가,등락률,거래변동량,코스피지수_증감량
0,2010-05-14,135837.5,0.000000,0.00,0.00
1,2010-05-17,136225.0,-0.473023,26597.00,-44.12
2,2010-05-18,140650.0,1.299365,43176.25,-8.27
3,2010-05-19,138175.0,-1.185370,28548.00,-13.16
4,2010-05-20,138462.5,-0.102268,29688.25,-29.90
...,...,...,...,...,...
2995,2022-07-06,197660.0,0.165308,24833.00,-49.77
2996,2022-07-07,197790.0,0.501700,41801.00,42.26
2997,2022-07-08,196530.0,-0.394951,23997.00,16.34
2998,2022-07-11,195770.0,-0.759630,16710.00,-10.34


# 7. 부동소수점 고정 & 타입 변환

In [32]:
for sector_name, df in result.items():
    df['종가'] = df['종가'].astype(int)
    df['등락률'] = df['등락률'].round(4)
    df['거래변동량'] =df['거래변동량'].astype(int)

    

In [35]:
for sector_name, df in result.items():
    df.set_index('일자',inplace=True)

In [36]:
result['Food']

,종가,등락률,거래변동량,코스피지수_증감량
일자,,,,
2010-05-14,135837,0.0000,0,0.00
2010-05-17,136225,-0.4730,26597,-44.12
2010-05-18,140650,1.2994,43176,-8.27
2010-05-19,138175,-1.1854,28548,-13.16
2010-05-20,138462,-0.1023,29688,-29.90
...,...,...,...,...
2022-07-06,197660,0.1653,24833,-49.77
2022-07-07,197790,0.5017,41801,42.26
2022-07-08,196530,-0.3950,23997,16.34


# 8. 정규화

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


df_scaled = {}
#스케일을 적용할 column을 정의합니다.
scale_cols = ['종가','등락률','거래변동량', '코스피지수_증감량']
for sector_name, df in result.items():
    scaler = MinMaxScaler()
    scaled=scaler.fit_transform(df[scale_cols])
    df_scaled[sector_name] = pd.DataFrame(scaled,index = df.index,columns=scale_cols)
    

In [41]:
df_scaled['Food']

,종가,등락률,거래변동량,코스피지수_증감량
일자,,,,
2010-05-14,0.074920,0.491660,0.000000,0.511587
2010-05-17,0.077067,0.468725,0.025468,0.342590
2010-05-18,0.101557,0.554667,0.041343,0.479910
2010-05-19,0.087859,0.434181,0.027336,0.461179
2010-05-20,0.089447,0.486699,0.028428,0.397058
...,...,...,...,...
2022-07-06,0.417075,0.499675,0.023779,0.320948
2022-07-07,0.417794,0.515987,0.040026,0.673459
2022-07-08,0.410821,0.472507,0.022978,0.574176


In [42]:
with open('df_scaled.pickle','wb') as f:
    pickle.dump(df_scaled,f)